<a href="https://colab.research.google.com/github/X-ray-Dawgz/XRayDawgz/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b rob https://github.com/X-ray-Dawgz/XRayDawgz.git

fatal: destination path 'XRayDawgz' already exists and is not an empty directory.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing import image

import numpy as np
import os

Using TensorFlow backend.


In [0]:
# image size
img_width, img_height = 432,288
# train and test sample sizes
train_samples = 665
test_samples = 98
# set weights
epochs = 75
batch_size = 20
# Directories of train and test data
train_dir = 'XRayDawgz/Images/Train'
test_dir = 'XRayDawgz/Images/Test'

In [0]:
# data image augmentation to create more variation of XRD patterns
train_data = ImageDataGenerator(rescale=1. / 255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

# only resize for test data
test_data = ImageDataGenerator(rescale=1. / 255)

In [23]:
# import train data consisting of 'binary' or  2 classifications (BCC or FCC) 
train_gen = train_data.flow_from_directory(train_dir, target_size=(img_width, img_height),
                                           batch_size=batch_size,
                                           class_mode='binary')

Found 1330 images belonging to 2 classes.


In [24]:
# import test data consisting of 'binary' or  2 classifications (BCC or FCC) 
test_gen = test_data.flow_from_directory(test_dir, target_size=(img_width, img_height),
                                           batch_size=batch_size,
                                           class_mode='binary')

Found 196 images belonging to 2 classes.


In [25]:
# build CNN model
model = Sequential()
# extract features by iterating across image
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(ZeroPadding2D((1,1)))
#convert 3D features to 1D feature vectors
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 430, 286, 32)      896       
_________________________________________________________________
activation_16 (Activation)   (None, 430, 286, 32)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 215, 143, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 213, 141, 32)      9248      
_________________________________________________________________
activation_17 (Activation)   (None, 213, 141, 32)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 106, 70, 32)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 104, 68, 64)      

In [0]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [27]:
model.fit_generator(train_gen,
                    steps_per_epoch=train_samples // batch_size,
                    epochs=epochs,
                    validation_data=test_gen,
                    validation_steps=test_samples // batch_size)

Epoch 1/75
33/33 [==============================] - 21s 643ms/step - loss: 5.7916 - acc: 0.5152 - val_loss: 0.7192 - val_acc: 0.4625
Epoch 2/75
33/33 [==============================] - 20s 601ms/step - loss: 0.6961 - acc: 0.4758 - val_loss: 0.6934 - val_acc: 0.4750
Epoch 3/75
33/33 [==============================] - 20s 616ms/step - loss: 0.6934 - acc: 0.5045 - val_loss: 0.6932 - val_acc: 0.4868
Epoch 4/75
33/33 [==============================] - 20s 608ms/step - loss: 0.7275 - acc: 0.4682 - val_loss: 0.6930 - val_acc: 0.4875
Epoch 5/75
33/33 [==============================] - 20s 608ms/step - loss: 0.7028 - acc: 0.4894 - val_loss: 0.6926 - val_acc: 0.4868
Epoch 6/75
33/33 [==============================] - 21s 624ms/step - loss: 0.6911 - acc: 0.5364 - val_loss: 0.6932 - val_acc: 0.5125
Epoch 7/75
33/33 [==============================] - 19s 590ms/step - loss: 0.6950 - acc: 0.4818 - val_loss: 0.6935 - val_acc: 0.4750
Epoch 8/75
33/33 [==============================] - 20s 612ms/step - 

In [54]:
BCC_test_dir = 'XRayDawgz/Images/Test/BCC_test/'
FCC_test_dir = 'XRayDawgz/Images/Test/FCC_test/'

BCC_Prediction = np.zeros(98)
FCC_Prediction = np.zeros(98)
i=0
correct_count=0
for file in os.listdir(BCC_test_dir):
  predict_img = image.load_img(BCC_test_dir + file, target_size = (img_width, img_height))
  predict_img = image.img_to_array(predict_img)
  predict_img = np.expand_dims(predict_img, axis = 0)
  result = model.predict(predict_img)
  pred = result.astype(int)
  if pred[0] is 0:
    correct_count += 1
  else:
    pass
print('BCC prediction accuracy: ' + str(correct_count // 98))

i=0
for file in os.listdir(FCC_test_dir):
  predict_img = image.load_img(FCC_test_dir + file, target_size = (img_width, img_height))
  predict_img = image.img_to_array(predict_img)
  predict_img = np.expand_dims(predict_img, axis = 0)
  result = model.predict(predict_img)
  FCC_Prediction[i] = result
  i += 1
print(FCC_Prediction)

BCC prediction accuracy: 0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1.]


In [20]:
predict_img = image.load_img('download.png', target_size = (img_width, img_height))
predict_img = image.img_to_array(predict_img)
predict_img = np.expand_dims(predict_img, axis = 0)

result = np.round(model.predict(predict_img), decimals=0)
print(np.round(result, decimals=0))

[[1.]]
